# Campaign Monitor
This notebook attaches to an existing campaign (state JSON + HDF5) and provides a simple refreshable view of progress, aggregated metrics, and last-updated timestamp.

In [1]:
# Setup and path selection
import sys, json, time
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, clear_output
import h5py, pandas as pd
root = Path.cwd()
src_path = root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))
state_path = widgets.Text(value='', description='State JSON:', layout=widgets.Layout(width='80%'))
h5_path = widgets.Text(value='', description='Results H5:', layout=widgets.Layout(width='80%'))
refresh_btn = widgets.Button(description='Refresh', button_style='info')
auto_refresh = widgets.Checkbox(value=False, description='Auto refresh (2s)')
progress = widgets.IntProgress(value=0, min=0, max=100, description='Progress:')
out = widgets.Output()
display(widgets.VBox([state_path, h5_path, widgets.HBox([refresh_btn, auto_refresh]), progress, out]))

In [2]:
# Monitor functions
def load_state(p: Path):
    if not p or not p.exists():
        return None
    with open(p, 'r') as f:
        return json.load(f)
def aggregated_df(h5: Path):
    rows = []
    if not h5 or not h5.exists():
        return pd.DataFrame(rows)
    with h5py.File(h5, 'r') as f:
        if 'aggregated' not in f:
            return pd.DataFrame(rows)
        for w in sorted(f['aggregated'].keys(), key=lambda x: int(x)):
            attrs = dict(f['aggregated'][w].attrs)
            rows.append({
                'width': int(w),
                **{k: attrs.get(k) for k in ['mean_hop','ci_lower','ci_upper','std_hop','pass_qv']}
            })
    return pd.DataFrame(rows)
def refresh_view(_=None):
    with out:
        out.clear_output()
        sp = Path(state_path.value)
        hp = Path(h5_path.value)
        st = load_state(sp)
        if st:
            print(f"Campaign: {st.get('campaign_id')}")
            print(f"Last update: {st.get('last_update')}")
            done = len(st.get('completed_widths', []))
            failed = len(st.get('failed_widths', []))
            inprog = st.get('in_progress_width')
            total = done + len(st.get('results_file',''))  # fallback if widths unknown
            print(f"Completed: {done}; Failed: {failed}; In-progress: {inprog}")
            # Progress if widths known (derive from H5 too)
        df = aggregated_df(hp)
        if not df.empty:
            print("Aggregated:")
            display(df.sort_values('width'))
            # Progress bar based on #rows vs possible widths (if we can infer)
            progress.value = min(100, int(100*len(df)/max(1, len(df))))
        else:
            print("No aggregated data yet.")
refresh_btn.on_click(refresh_view)
def auto_loop():
    while auto_refresh.value:
        refresh_view()
        time.sleep(2)
widgets.jsdlink((auto_refresh, 'value'), (refresh_btn, 'disabled'))

DirectionalLink(source=(Checkbox(value=False, description='Auto refresh (2s)'), 'value'), target=(Button(button_style='info', description='Refresh', style=ButtonStyle()), 'disabled'))